<a href="https://colab.research.google.com/github/ArtyomShabunin/SMOPA-25/blob/main/lesson_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://prana-system.com/files/110/rds_color_full.png" alt="tot image" width="300"  align="center"/> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src="https://mpei.ru/AboutUniverse/OficialInfo/Attributes/PublishingImages/logo1.jpg" alt="mpei image" width="200" align="center"/>
<img src="https://mpei.ru/Structure/Universe/tanpe/structure/tfhe/PublishingImages/tot.png" alt="tot image" width="100"  align="center"/>

---

# **Системы машинного обучения и предиктивной аналитики в тепловой и возобновляемой энергетике**  

# ***Практические занятия***


---

# Занятие №7
# Многоклассовая классификация методами глубокого обучения
**2 апреля 2025г.**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler

from imblearn.under_sampling import RandomUnderSampler

from tqdm import tqdm
import json

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')
device

## Загрузка данных

In [ ]:
# import gdown
# import warnings
# warnings.filterwarnings('ignore')
# gdown.download('https://drive.google.com/uc?id=1j54o4pHTm3HvaYTEtv_i4hOJGy5yNeZZ', verify=False)

data = pd.read_parquet("./data_modes.gzip")

In [ ]:
data.head()

Чтение файла с описанием сигналов

In [ ]:
# import gdown
# url = "https://drive.google.com/drive/folders/1RtrAevJUYSgTbp0YUztxEBB8_VcvjgGH?usp=drive_link"
# gdown.download_folder(url, quiet=True, verify=False)

with open(f'./option_0/description.json', 'r', encoding = "utf-8") as f:
    description = json.load(f)

Составим словарь для трактовки наименований сигналов

In [ ]:
kks_to_description = {param['real_kks']: f"{param['description']}, [{param['unit']}]"
for param in description if param['real_kks'] in data.columns}

description_to_kks = { f"{param['description']}, [{param['unit']}]": param['real_kks']
for param in description if param['real_kks'] in data.columns}

## Сформируем датасет для решения задачи многоклассовой классификации

Выберем сигналы которые будем использовать. Исключим все признаки на основании которых выделялись целевые режимы.

In [ ]:
feature_columns = [
    'GTA1.DBinPU.Alzzo', 'GTA1.DBinPU.Bo', 'GTA1.DBinPU.DlPkf',
    'GTA1.DBinPU.DlPtgft', 'GTA1.DBinPU.DlPvf', 'GTA1.DBinPU.fi',
    'GTA1.DBinPU.hmGTD', 'GTA1.DBinPU.hmTG', 'GTA1.DBinPU.P1mvhTG',
    'GTA1.DBinPU.Pk',
    'GTA1.DBinPU.Pmvh', 'GTA1.DBinPU.PmvhMOGTD',
    'GTA1.DBinPU.PmvhMOTG', 'GTA1.DBinPU.PmvyhMOGTD',
    'GTA1.DBinPU.PmvyhMOTG', 'GTA1.DBinPU.Prazrjag_navhode',
    'GTA1.DBinPU.Ptgpd', 'GTA1.DBinPU.Ptgvh',
    'GTA1.DBinPU.Pvh',
    'GTA1.DBinPU.Pvyhlg',
    'GTA1.DBinPU.Qtg',
    'GTA1.DBinPU.Tk',
    'GTA1.DBinPU.Tn', 'GTA1.DBinPU.Tt', 'GTA1.DBinPU.Tvh1',
    'GTA1.DBinPU.Pzad'
    ]

target_columns = [
    'full_power_mode',
    'partial_power_mode',
    'increas_power_mode',
    'decreas_power_mode',
    'start_up_mode',
    'shutdown_mode',
    'stopped_state_mode'
]

В данных прсутствуют примеры которые одновременно относятся к нескольким режимам. Класс таких задач называется **многоклассовой классификацией с пересечением классов** (*multi-label classification*). Нам необходимо избавиться от тких примеров.

In [ ]:
data[data[target_columns].sum(axis=1) > 1][target_columns]

In [ ]:
data.loc[data['adjustment_range'], ['start_up_mode']] = False
data.loc[data['adjustment_range'], ['shutdown_mode']] = False
data.loc[data['increase_power'], ['shutdown_mode']] = False
data.loc[data['decreas_power_mode'], ['start_up_mode']] = False
data.loc[data['stopped_state_mode'], ['start_up_mode']] = False

In [ ]:
data[data[target_columns].sum(axis=1) > 1][target_columns].shape

In [ ]:
data = data.loc[data[target_columns].sum(axis=1) == 1]

Создадим датафрейм с признаками и целевыми значения

In [ ]:
X = data.loc[:,feature_columns]

In [ ]:
data['target'] = data[target_columns].idxmax(axis=1)
y = data.loc[:, ['target']]

In [ ]:
y.value_counts()

### Деление на тестовую и тренировочную выборки

In [ ]:
from sklearn.model_selection import train_test_split
# Разделяем с учетом дисбаланса классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

Видно, что датасет сильно несбалансирован.
При такой несбалансорованности модели будут плохо обучаеться на редких классах.
Алгоритм может игнорировать малочисленные классы, потому что минимизация ошибки будет достигаться за счет предсказания большинства.
Например, если модель всегда предсказывает full_power_mode, она все равно получит высокую точность.

### Балансировка данных
**Oversampling (увеличение малых классов)**  
   - Повторение существующих редких примеров или их генерация.  
   - **SMOTE (Synthetic Minority Over-sampling Technique)** – создает новые точки малочисленных классов, используя линейные комбинации соседних точек.  

**Undersampling (уменьшение частых классов)**  
- Удаление случайных примеров из большинства классов.  

**Комбинация Oversampling + Undersampling**  
   - Часто лучше сначала **уменьшить большие**, а затем **увеличить малые классы**.  

#### Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
sampling_strategy = {
    "full_power_mode": 1000,
    "stopped_state_mode": 1000,
    "partial_power_mode": 1000
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
y_train_resampled.value_counts()

In [ ]:
sampling_strategy = {
    "full_power_mode": 100,
    "stopped_state_mode": 100,
    "partial_power_mode": 100
}
rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=42)
X_test_resampled, y_test_resampled = rus.fit_resample(X_test, y_test)

In [ ]:
y_test_resampled.value_counts()

#### Oversampling

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_resampled, y_train_resampled)

In [ ]:
y_train_resampled.value_counts()

### Нормализация или стандартизация данных

In [ ]:
# scaler = preprocessing.MinMaxScaler() # нормализация данных
scaler = preprocessing.StandardScaler() # стандартизация данных

X_train_resampled_scaled = pd.DataFrame(
    scaler.fit_transform(X_train_resampled),
    columns=X_train_resampled.columns,
    index=X_train_resampled.index)

X_test_resampled_scaled = pd.DataFrame(
    scaler.transform(X_test_resampled),
    columns=X_test_resampled.columns,
    index=X_test_resampled.index)

X_train_resampled_scaled.describe()

### LabelEncoding

Закодируем текстовые метки классов в числовые значения

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
y_train_resampled_encoded = encoder.fit_transform(y_train_resampled.values[:,0])
y_test_resampled_encoded = encoder.transform(y_test_resampled.values[:,0])

In [ ]:
y_test_resampled_encoded

In [ ]:
encoder.classes_

### Dataset и DataLoader

`Dataset` и `DataLoader` — это ключевые классы в PyTorch для работы с данными при обучении нейросетей.

`Dataset` — это абстрактный класс для работы с данными. Он определяет, как загружаются данные, а также как они хранятся и индексируются. Все датасеты в PyTorch должны наследоваться от `torch.utils.data.Dataset` и реализовывать два метода:
- `__len__()`: возвращает количество элементов в датасете.
- `__getitem__(index)`: позволяет по индексу получить один элемент данных.

---

`DataLoader` — это инструмент для удобной загрузки данных партиями (батчами) во время обучения модели. Он берет объект `Dataset` и выполняет:
- разбиение данных на батчи (`batch_size`),
- перемешивание (`shuffle`),
- многопоточное извлечение данных (`num_workers`).

---

### Итог:
- `Dataset` отвечает за хранение и доступ к данным.
- `DataLoader` управляет загрузкой данных, разбиением на батчи и параллельной обработкой.

Эти классы делают работу с данными удобной и эффективной, особенно при обучении нейросетей.

In [ ]:
from torch.utils.data import Dataset

class ModeDataset(Dataset):
    def __init__(self):
        self.x = torch.tensor(X_train_resampled_scaled.values, dtype=torch.float32)
        self.y = torch.tensor(y_train_resampled_encoded, dtype=torch.long)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.x.shape[0]

In [ ]:
dataset = ModeDataset()

print(len(dataset))
print("Признаки №0 : ", dataset[0][0])
print("Целевая метка №0 : ", dataset[0][1])

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 256

INPUT_SIZE = len(feature_columns)
OUTPUT_SIZE = len(target_columns)

train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [train_size, valid_size])


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)

## Функция для обучения модели

**Что такое Learning Rate?**  

**Learning Rate (скорость обучения, $\eta$)** – это гиперпараметр, определяющий, **насколько сильно обновляются веса нейросети на каждом шаге градиентного спуска**.  

Если **learning rate слишком большой** → модель может "перепрыгивать" оптимальные веса и не сойтись.  
Если **learning rate слишком маленький** → обучение будет слишком медленным или застрянет в локальном минимуме.  

Пример обновления весов с learning rate $\eta$:  
$$w = w - \eta \cdot \frac{\partial L}{\partial w}$$

где:  
- $ w $ — веса модели,  
- $ \frac{\partial L}{\partial w} $ — градиент функции ошибки $ L $,  
- $ \eta $ — learning rate.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaK3GNwACWpqsTkfBvKdB9osthN7aWVE1ahw&s" alt="rf image" width="600"  align="center"/>

---

В функции `train_model` объекты `criterion`, `optimizer` и `scheduler` — это стандартные компоненты процесса обучения нейросетей в PyTorch, которые управляют процессом обучения модели:

1. **loss_function** — это функция потерь.  
   Она вычисляет, насколько предсказания модели отличаются от реальных значений. Например:
   - `nn.MSELoss()` — среднеквадратичная ошибка (используется в регрессии).
   - `nn.CrossEntropyLoss()` — кросс-энтропия (используется в классификации).

---

2. **optimizer** — это оптимизатор, который обновляет веса модели на основе градиентов, вычисленных через `loss.backward()`.  
   Примеры оптимизаторов:
   - `torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)` — стохастический градиентный спуск.
   - `torch.optim.Adam(model.parameters(), lr=0.001)` — адаптивный оптимизатор Adam.


Adam (**Adaptive Moment Estimation**) — один из самых популярных оптимизаторов в глубоком обучении.  

Преимущества Adam:
- Адаптивное изменение скорости обучения (learning rate)
- Быстро сходится к минимуму
- Работает хорошо без ручной настройки
- Устойчив к шуму и негладким функциям потерь
- Хорошо работает с разреженными данными

---

4. **scheduler** — это планировщик (scheduler) изменения скорости обучения (learning rate).  
   Он управляет уменьшением `lr` в зависимости от метрики (например, от значения функции потерь на обучающей выборке).  

**Зачем нужен scheduler?**  

Обычно в начале обучения выгодно использовать **большой** `learning rate` (чтобы быстро находить хорошие параметры), а к концу — **уменьшать** его, чтобы модель точнее подстраивалась.  

**Scheduler** – это инструмент, который автоматически изменяет learning rate **по заданному правилу**. Это помогает избежать проблем с застреванием или нестабильностью обучения.  



**Популярные стратегии изменения Learning Rate (Schedulers)**  
**StepLR** – уменьшает `lr` через заданное количество эпох.  
**ExponentialLR** – уменьшает `lr` экспоненциально.  
**ReduceLROnPlateau** – уменьшает `lr`, если метрика не улучшается.  
**CosineAnnealingLR** – снижает `lr` по косинусной функции, часто используется в глубоких сетях.  

Использование **learning rate scheduler'а** позволяет сделать обучение модели **более эффективным** и **устойчивым**.

Использование **scheduler** (планировщика скорости обучения) при **Adam** — не всегда обязательно, но может улучшить обучение модели.


In [ ]:
def train_model(model, loss_function, optimizer, scheduler, num_epochs=100):

    loaders = {"train": train_loader, "valid": valid_loader}

    epochs = num_epochs

    lr = []
    losses = {"train": [], "valid": []}
    for epoch in tqdm(range(epochs)):

        for k, dataloader in loaders.items():
            running_loss = []

            for x_batch, y_batch in dataloader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                if k == "train":
                    model.train()
                    optimizer.zero_grad()
                    outp = model(x_batch)
                else:
                    model.eval()
                    with torch.no_grad():
                        outp = model(x_batch)

                loss = loss_function(outp, y_batch)
                running_loss.append(loss.item())

                if k == "train":
                    loss.backward()
                    optimizer.step()

            if k == "train":
                lr.append(scheduler.optimizer.param_groups[0]['lr'])
            losses[k].append(np.array(running_loss).mean())

        scheduler.step(losses["train"][-1])

    return model, losses, lr

## Полносвязная нейронная сеть (многослойный персептрон)

<img src="https://miro.medium.com/v2/resize:fit:720/1*VHOUViL8dHGfvxCsswPv-Q.png" alt="rf image" width="600"  align="center"/>



## Обучение моделей

Инициализируем переменные для дальнейшего сравнения моделей

In [ ]:
accuracy_classifier = {}
precision_classifier = {}
recall_classifier = {}
f1_classifier = {}

### Модель №1

Простая полносвязная сеть.  
В PyTorch нейронная сеть создается как класс, унаследованный от `torch.nn.Module`. Давай рассмотрим пошагово, как это делается.  

- В `__init__` определяем слои нейросети.  
- В `forward` описываем, как данные проходят через слои.  



- `nn.Linear` — это полносвязный (линейный) слой в PyTorch. Он выполняет линейное преобразование входных данных:  

$$ y = xW^T + b $$

Где:  
$ x $ — входной тензор (данные),  
$ W $ — обучаемые веса слоя,  
$ b $ — обучаемые смещения (bias),  
$ y $ — выходные значения.  

- `nn.Relu()` — функция активации ReLU.
- `nn.Sequential` объединяет несколько слоев в последовательность, через которую данные проходят автоматически (без явного вызова `forward`).
- `nn.ModuleList` — это просто список (`list`) для хранения слоев, но без автоматического выполнения `forward`. Это полезно, когда требуется сложная логика.

In [ ]:
class FCNN(nn.Module):
  def __init__(
      self, hidden_size=512, hidden_num=1):
    super(FCNN, self).__init__()

    # Входной слой
    self.input_layer = nn.Sequential(
        nn.Linear(INPUT_SIZE, hidden_size),
        nn.ReLU(),
    )

    # Скрытые слои
    self.hidden_layers = nn.ModuleList()
    for _ in range(hidden_num):
        self.hidden_layers.append(
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.ReLU(),
            )
        )

    # Выходной слой
    self.output_layer = nn.Linear(hidden_size, OUTPUT_SIZE)

  def forward(self, x):
    x = self.input_layer(x)

    for layer in self.hidden_layers:
        x = layer(x)

    x = self.output_layer(x)
    return x


#### Обучение модели

**CrossEntropyLoss**  

`CrossEntropyLoss` – это функция потерь, которая используется в задачах **многоклассовой классификации**. Она измеряет, насколько предсказанное распределение вероятностей отличается от истинных классов.  

**Формула CrossEntropyLoss:**  
Для одного объекта:
$$\text{Loss} = - \sum_{i=1}^{C} y_i \log(\hat{p}_i)$$
где:  
- $ C $ – количество классов,  
- $ y_i $ – индикатор (0 или 1), показывающий правильный класс,  
- $ \hat{p}_i $ – предсказанная вероятность для класса $ i $.  

Если правильный класс – это $ k $, то формула упрощается до:  
$$\text{Loss} = -\log(\hat{p}_k)$$
Это значит, что штрафуется именно вероятность правильного класса – чем она ниже, тем выше ошибка.  

---

**Особенности:**
1. **Работает с "сырыми" логитами (до softmax).** Внутри себя `CrossEntropyLoss` сама применяет `softmax`, так что передавать уже нормализованные вероятности **не нужно**.  
2. **Подходит для one-hot меток, но чаще используется с индексами классов.**  
3. **Используется в многоклассовой классификации** (если у примера один правильный класс).

---

**Softmax**  

`Softmax` – это функция, которая **преобразует логиты** (сырые выходы нейросети) в **вероятности**, так чтобы их сумма была **равна 1**. Она часто используется в задачах многоклассовой классификации.  

**Формула Softmax:**  
Для каждого выхода $ z_i $ нейросети:  
$$\hat{p}_i = \frac{e^{z_i}}{\sum_{j=1}^{C} e^{z_j}}$$  
где:  
- $ \hat{p}_i $ – вероятность, что объект принадлежит классу $ i $,  
- $ C $ – количество классов,  
- $ z_i $ – логит для класса $ i $,  
- $ e^{z_i} $ – экспоненциальное преобразование логита (делает все значения положительными).  

**Что делает Softmax?**
1. **Нормализует выходы нейросети** – переводит их в диапазон $ [0,1] $.  
2. **Гарантирует, что сумма всех выходов = 1** (их можно интерпретировать как вероятности).  
3. **Подчеркивает разницу между выходами** – чем больше разница между логитами, тем сильнее выделяется наиболее вероятный класс.  

In [ ]:
FCNN_16_2 = FCNN(16, 2).to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(
    FCNN_16_2.parameters(), lr=1e-3, weight_decay=1e-5)

scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,
                                           patience=30, threshold=0.0001)

FCNN_16_2, losses_FCNN_16_2, lr_FCNN_16_2 = train_model(
    FCNN_16_2, loss_function, optimizer,
    scheduler, num_epochs=200)

Посмотрим на график изменения среднего на эпохе значения функции потерь.

In [ ]:
plt.plot(losses_FCNN_16_2['train'], label='train');
plt.plot(losses_FCNN_16_2['valid'], label='valid');
plt.legend();
plt.ylabel("Ошибка");
plt.xlabel("Эпоха обучения");

In [ ]:
plt.plot(lr_FCNN_16_2);
plt.ylabel("learning rate");
plt.xlabel("Эпоха обучения");

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {losses_FCNN_16_2["train"][-1]:.5f}')
print(f'На валидационной выборке: {losses_FCNN_16_2["valid"][-1]:.5f}')

Посмотрим предсказания на нескольких случайных примерах.


In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

FCNN_16_2.to("cpu")
FCNN_16_2.eval()

logits = FCNN_16_2(torch.tensor(X_some_modes.values, dtype=torch.float32))
predicted_classes = torch.argmax(logits, dim=1)

for target, predict in zip(y_some_modes.values, predicted_classes.detach().numpy()):
    print(f"Истина - {target[0]} >>> {encoder.inverse_transform([predict])[0]} - предсказание")

Посмотрим на уверенность модели в своих предсказаниях.

In [ ]:
pd.DataFrame(torch.softmax(logits, dim=1).detach().numpy(), columns=encoder.classes_).map(lambda x: f'{x:.3f}')

#### Анализ качества модели

In [ ]:
FCNN_16_2.eval()
logits =  FCNN_16_2(torch.tensor(X_test_resampled_scaled.values, dtype=torch.float32))

probabilities = torch.softmax(logits, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

y_test_pred_FCNN_16_2 = encoder.inverse_transform(predicted_classes)

**Матрица неточностей**

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_FCNN_16_2)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

**Accuracy**

In [ ]:
accuracy_classifier['FCNN_16_2'] = accuracy_score(y_test_resampled, y_test_pred_FCNN_16_2)

**Precision и recall**

In [ ]:
precision_classifier['FCNN_16_2'] = precision_score(y_test_resampled, y_test_pred_FCNN_16_2, average='macro', zero_division = np.nan)
recall_classifier['FCNN_16_2'] = recall_score(y_test_resampled, y_test_pred_FCNN_16_2, average='macro', zero_division = np.nan)

**F1**

In [ ]:
f1_classifier['FCNN_16_2'] = f1_score(y_test_resampled, y_test_pred_FCNN_16_2, average='macro', zero_division = np.nan)

**Значения метрик**

In [ ]:
print(f"accuracy - {accuracy_classifier['FCNN_16_2']*100:0.2f}%")
print(f"precision - {precision_classifier['FCNN_16_2']*100:0.2f}%")
print(f"recall - {recall_classifier['FCNN_16_2']*100:0.2f}%")
print(f"F1 - {f1_classifier['FCNN_16_2']*100:0.2f}%")

### Модель №2
Увеличим число скрытых слоев в модели
#### Обучение модели

In [ ]:
FCNN_16_4 = FCNN(16, 4).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(FCNN_16_4.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,
                                           patience=30, threshold=0.0001)

FCNN_16_4, losses_FCNN_16_4, lr_FCNN_16_4 = train_model(
    FCNN_16_4, loss_function, optimizer,
    scheduler, num_epochs=200)

Посмотрим на график изменения среднего на эпохе значения функции потерь.

In [ ]:
plt.plot(losses_FCNN_16_4['train'], label='train');
plt.plot(losses_FCNN_16_4['valid'], label='valid');
plt.legend();
plt.ylabel("Ошибка");
plt.xlabel("Эпоха обучения");

In [ ]:
plt.plot(lr_FCNN_16_4);
plt.ylabel("learning rate");
plt.xlabel("Эпоха обучения");

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {losses_FCNN_16_4["train"][-1]:.5f}')
print(f'На валидационной выборке: {losses_FCNN_16_4["valid"][-1]:.5f}')

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

FCNN_16_4.to("cpu")
FCNN_16_4.eval()
logits = FCNN_16_4(torch.tensor(X_some_modes.values, dtype=torch.float32))
predicted_classes = torch.argmax(logits, dim=1)

for target, predict in zip(y_some_modes.values, predicted_classes.detach().numpy()):
    print(f"Истина - {target[0]} >>> {encoder.inverse_transform([predict])[0]} - предсказание")

Посмотрим на уверенность модели в своих предсказаниях.

In [ ]:
pd.DataFrame(torch.softmax(logits, dim=1).detach().numpy(), columns=encoder.classes_).map(lambda x: f'{x:.3f}')

#### Анализ качества модели

In [ ]:
FCNN_16_4.eval()
logits =  FCNN_16_4(torch.tensor(X_test_resampled_scaled.values, dtype=torch.float32))

probabilities = torch.softmax(logits, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

y_test_pred_FCNN_16_4 = encoder.inverse_transform(predicted_classes)

**Матрица неточностей**

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_FCNN_16_4)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

**Accuracy**

In [ ]:
accuracy_classifier['FCNN_16_4'] = accuracy_score(y_test_resampled, y_test_pred_FCNN_16_4)

**Precision и recall**

In [ ]:
precision_classifier['FCNN_16_4'] = precision_score(y_test_resampled, y_test_pred_FCNN_16_4, average='macro', zero_division = np.nan)
recall_classifier['FCNN_16_4'] = recall_score(y_test_resampled, y_test_pred_FCNN_16_4, average='macro', zero_division = np.nan)

**F1**

In [ ]:
f1_classifier['FCNN_16_4'] = f1_score(y_test_resampled, y_test_pred_FCNN_16_4, average='macro', zero_division = np.nan)

**Значения метрик**

In [ ]:
print(f"accuracy - {accuracy_classifier['FCNN_16_4']*100:0.2f}%")
print(f"precision - {precision_classifier['FCNN_16_4']*100:0.2f}%")
print(f"recall - {recall_classifier['FCNN_16_4']*100:0.2f}%")
print(f"F1 - {f1_classifier['FCNN_16_4']*100:0.2f}%")

### Модель №3
Увеличим размер скрытых слоев в модели
### Обучение модели

In [ ]:
FCNN_32_4 = FCNN(32, 4).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(FCNN_32_4.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,
                                           patience=30, threshold=0.0001)

FCNN_32_4, losses_FCNN_32_4, lr_FCNN_32_4 = train_model(
    FCNN_32_4, loss_function, optimizer,
    scheduler, num_epochs=200)

Посмотрим на график изменения среднего на эпохе значения функции потерь.

In [ ]:
plt.plot(losses_FCNN_32_4['train'], label='train');
plt.plot(losses_FCNN_32_4['valid'], label='valid');
plt.legend();
plt.ylabel("Ошибка");
plt.xlabel("Эпоха обучения");

In [ ]:
plt.plot(lr_FCNN_32_4);
plt.ylabel("learning rate");
plt.xlabel("Эпоха обучения");

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {losses_FCNN_32_4["train"][-1]:.5f}')
print(f'На валидационной выборке: {losses_FCNN_32_4["valid"][-1]:.5f}')

Посмотрим предсказания на нескольких случайных примерах.

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

FCNN_32_4.to("cpu")
FCNN_32_4.eval()
logits = FCNN_32_4(torch.tensor(X_some_modes.values, dtype=torch.float32))
predicted_classes = torch.argmax(logits, dim=1)

for target, predict in zip(y_some_modes.values, predicted_classes.detach().numpy()):
    print(f"Истина - {target[0]} >>> {encoder.inverse_transform([predict])[0]} - предсказание")

Посмотрим на уверенность модели в своих предсказаниях.

In [ ]:
pd.DataFrame(torch.softmax(logits, dim=1).detach().numpy(), columns=encoder.classes_).applymap(lambda x: f'{x:.3f}')

#### Анализ качества модели

In [ ]:
FCNN_32_4.eval()
logits =  FCNN_32_4(torch.tensor(X_test_resampled_scaled.values, dtype=torch.float32))

probabilities = torch.softmax(logits, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

y_test_pred_FCNN_32_4 = encoder.inverse_transform(predicted_classes)

**Матрица неточностей**

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_FCNN_32_4)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

**Accuracy**

In [ ]:
accuracy_classifier['FCNN_32_4'] = accuracy_score(y_test_resampled, y_test_pred_FCNN_32_4)

**Precision и recall**

In [ ]:
precision_classifier['FCNN_32_4'] = precision_score(y_test_resampled, y_test_pred_FCNN_32_4, average='macro', zero_division = np.nan)
recall_classifier['FCNN_32_4'] = recall_score(y_test_resampled, y_test_pred_FCNN_32_4, average='macro', zero_division = np.nan)

**F1**

In [ ]:
f1_classifier['FCNN_32_4'] = f1_score(y_test_resampled, y_test_pred_FCNN_32_4, average='macro', zero_division = np.nan)

**Значения метрик**

In [ ]:
print(f"accuracy - {accuracy_classifier['FCNN_32_4']*100:0.2f}%")
print(f"precision - {precision_classifier['FCNN_32_4']*100:0.2f}%")
print(f"recall - {recall_classifier['FCNN_32_4']*100:0.2f}%")
print(f"F1 - {f1_classifier['FCNN_32_4']*100:0.2f}%")

### Модель №4

**Batch Normalization (BatchNorm)**  

**Batch Normalization (нормализация по батчам, BatchNorm)** – это метод, который стабилизирует и ускоряет обучение нейросетей, **нормализуя входные данные на каждом слое**.  

**Главная идея:**  
Каждый слой нейросети получает входные данные, которые могут сильно различаться по масштабу. BatchNorm **выравнивает их распределение**, приводя к более стабильному градиентному спуску.  

---

**Зачем нужен BatchNorm?**  
**Стабилизирует градиентный спуск** (предотвращает "взрыв" или "затухание" градиентов).  
**Позволяет использовать более высокий `learning rate`**, ускоряя обучение.  
**Уменьшает зависимость от инициализации весов**.  
**Добавляет небольшой эффект регуляризации** (схожий с Dropout).  


---

**Когда использовать BatchNorm?**  
В **глубоких нейросетях**, чтобы улучшить сходимость.  
В **сверточных нейросетях (CNN)** для стабилизации обучения.  
В **полносвязных сетях (MLP)**, если признаки сильно различаются по масштабу.  

Не нужен в **маленьких сетях** или **если уже используется LayerNorm / GroupNorm**.


In [ ]:
class FCNN_BN(nn.Module):
  def __init__(
      self, hidden_size=512, hidden_num=1,
      # dropout_rate=0
      ):
    super(FCNN_BN, self).__init__()

    # Входной слой
    self.input_layer = nn.Sequential(
        nn.Linear(INPUT_SIZE, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(),
    )

    # Скрытые слои
    self.hidden_layers = nn.ModuleList()
    for _ in range(hidden_num):
        self.hidden_layers.append(
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
            )
        )

    # Выходной слой
    self.output_layer = nn.Linear(hidden_size, OUTPUT_SIZE)

  def forward(self, x):
    x = self.input_layer(x)

    for layer in self.hidden_layers:
        x = layer(x)

    x = self.output_layer(x)
    return x

#### Обучение модели

In [ ]:
FCNN_BN_32_4 = FCNN_BN(32, 4).to(device)

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(FCNN_BN_32_4.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,
                                           patience=30, threshold=0.0001)

FCNN_BN_32_4, losses_FCNN_BN_32_4, lr_FCNN_BN_32_4 = train_model(
    FCNN_BN_32_4, loss_function, optimizer,
    scheduler, num_epochs=200)

Посмотрим на график изменения среднего на эпохе значения функции потерь.

In [ ]:
plt.plot(losses_FCNN_BN_32_4['train'], label='train');
plt.plot(losses_FCNN_BN_32_4['valid'], label='valid');
plt.legend();
plt.ylabel("Ошибка");
plt.xlabel("Эпоха обучения");

In [ ]:
plt.plot(lr_FCNN_BN_32_4);
plt.ylabel("learning rate");
plt.xlabel("Эпоха обучения");

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {losses_FCNN_BN_32_4["train"][-1]:.5f}')
print(f'На валидационной выборке: {losses_FCNN_BN_32_4["valid"][-1]:.5f}')

Посмотрим предсказания на нескольких случайных примерах.

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

FCNN_BN_32_4.to("cpu")
FCNN_BN_32_4.eval()
logits = FCNN_BN_32_4(torch.tensor(X_some_modes.values, dtype=torch.float32))
predicted_classes = torch.argmax(logits, dim=1)

for target, predict in zip(y_some_modes.values, predicted_classes.detach().numpy()):
    print(f"Истина - {target[0]} >>> {encoder.inverse_transform([predict])[0]} - предсказание")

Посмотрим на уверенность модели в своих предсказаниях.

In [ ]:
pd.DataFrame(torch.softmax(logits, dim=1).detach().numpy(), columns=encoder.classes_).map(lambda x: f'{x:.3f}')

#### Анализ качества модели

In [ ]:
FCNN_BN_32_4.eval()
logits =  FCNN_BN_32_4(
    torch.tensor(X_test_resampled_scaled.values, dtype=torch.float32))

probabilities = torch.softmax(logits, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

y_test_pred_FCNN_BN_32_4 = encoder.inverse_transform(predicted_classes)

**Матрица неточностей**

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_FCNN_BN_32_4)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

**Accuracy**

In [ ]:
accuracy_classifier['FCNN_BN_32_4'] = accuracy_score(y_test_resampled, y_test_pred_FCNN_BN_32_4)

**Precision и recall**

In [ ]:
precision_classifier['FCNN_BN_32_4'] = precision_score(y_test_resampled, y_test_pred_FCNN_BN_32_4, average='macro', zero_division = np.nan)
recall_classifier['FCNN_BN_32_4'] = recall_score(y_test_resampled, y_test_pred_FCNN_BN_32_4, average='macro', zero_division = np.nan)

**F1**

In [ ]:
f1_classifier['FCNN_BN_32_4'] = f1_score(y_test_resampled, y_test_pred_FCNN_BN_32_4, average='macro', zero_division = np.nan)

**Значения метрик**

In [ ]:
print(f"accuracy - {accuracy_classifier['FCNN_BN_32_4']*100:0.2f}%")
print(f"precision - {precision_classifier['FCNN_BN_32_4']*100:0.2f}%")
print(f"recall - {recall_classifier['FCNN_BN_32_4']*100:0.2f}%")
print(f"F1 - {f1_classifier['FCNN_BN_32_4']*100:0.2f}%")

### Модель №5

**Dropout**  

`Dropout` – это **регуляризация**, которая помогает **предотвратить переобучение** нейросетей, случайно **"отключая" нейроны** во время обучения.  

**Идея:**  
Во время обучения с **некоторой вероятностью (p)** нейроны временно отключаются (их выходы становятся нулями). Это заставляет модель **не зависеть слишком сильно от отдельных нейронов** и делать более **устойчивые обобщения**.  

---

### **Как работает Dropout?**
1. **Во время обучения:**  
   - Для каждого нейрона с вероятностью $ p $ зануляется его выход.  
   - Оставшиеся нейроны работают с увеличенной силой $ 1/(1-p) $, чтобы компенсировать потери.  

2. **Во время инференса (предсказаний):**  
   - Dropout **не применяется** – все нейроны работают нормально.  
   - Но так как во время обучения активные нейроны усиливались, их выход остается неизменным.  

In [ ]:
class FCNN_BN_DO(nn.Module):
  def __init__(
      self, hidden_size=512, hidden_num=1,
      dropout_rate=0.5
      ):
    super(FCNN_BN_DO, self).__init__()

    # Входной слой
    self.input_layer = nn.Sequential(
        nn.Linear(INPUT_SIZE, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(),
        nn.Dropout(dropout_rate)
    )

    # Скрытые слои
    self.hidden_layers = nn.ModuleList()
    for _ in range(hidden_num):
        self.hidden_layers.append(
            nn.Sequential(
                nn.Linear(hidden_size, hidden_size),
                nn.BatchNorm1d(hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            )
        )

    # Выходной слой
    self.output_layer = nn.Linear(hidden_size, OUTPUT_SIZE)

  def forward(self, x):
    x = self.input_layer(x)

    for layer in self.hidden_layers:
        x = layer(x)

    x = self.output_layer(x)
    return x

#### Обучение модели

In [ ]:
FCNN_BN_DO_32_4 = FCNN_BN_DO(32, 4, 0.5).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    FCNN_BN_DO_32_4.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3,
                                           patience=30, threshold=0.0001)

FCNN_BN_DO_32_4, losses_FCNN_BN_DO_32_4, lr_FCNN_BN_DO_32_4 = train_model(
    FCNN_BN_DO_32_4, criterion, optimizer,
    scheduler, num_epochs=200)

Посмотрим на график изменения среднего на эпохе значения функции потерь.

In [ ]:
plt.plot(losses_FCNN_BN_DO_32_4['train'], label='train');
plt.plot(losses_FCNN_BN_DO_32_4['valid'], label='valid');
plt.legend();
plt.ylabel("Ошибка");
plt.xlabel("Эпоха обучения");

In [ ]:
plt.plot(lr_FCNN_BN_DO_32_4);
plt.ylabel("learning rate");
plt.xlabel("Эпоха обучения");

In [ ]:
print('Loss в конце обучения')
print(f'На обучающей выборке: {losses_FCNN_BN_DO_32_4["train"][-1]:.5f}')
print(f'На валидационной выборке: {losses_FCNN_BN_DO_32_4["valid"][-1]:.5f}')

Посмотрим предсказания на нескольких случайных примерах.

In [ ]:
some_indexes = y_test_resampled.groupby('target').sample(n=1, random_state=42).index
X_some_modes = X_test_resampled_scaled.loc[some_indexes]
y_some_modes = y_test_resampled.loc[some_indexes]

FCNN_BN_DO_32_4.to("cpu")
FCNN_BN_DO_32_4.eval()
logits = FCNN_BN_DO_32_4(torch.tensor(X_some_modes.values, dtype=torch.float32))
predicted_classes = torch.argmax(logits, dim=1)

for target, predict in zip(y_some_modes.values, predicted_classes.detach().numpy()):
    print(f"Истина - {target[0]} >>> {encoder.inverse_transform([predict])[0]} - предсказание")

Посмотрим на уверенность модели в своих предсказаниях.

In [ ]:
pd.DataFrame(torch.softmax(logits, dim=1).detach().numpy(), columns=encoder.classes_).map(lambda x: f'{x:.3f}')

#### Анализ качества модели

In [ ]:
FCNN_BN_DO_32_4.eval()
logits =  FCNN_BN_DO_32_4(
    torch.tensor(X_test_resampled_scaled.values, dtype=torch.float32))

probabilities = torch.softmax(logits, dim=1)
predicted_classes = torch.argmax(probabilities, dim=1)

y_test_pred_FCNN_BN_DO_32_4 = encoder.inverse_transform(predicted_classes)

**Матрица неточностей**

In [ ]:
conf_mat = confusion_matrix(y_test_resampled, y_test_pred_FCNN_BN_DO_32_4)
ConfusionMatrixDisplay(conf_mat, display_labels=encoder.classes_).plot()
plt.xticks(rotation=90)
plt.show()

**Accuracy**

In [ ]:
accuracy_classifier['FCNN_BN_DO_32_4'] = accuracy_score(
    y_test_resampled, y_test_pred_FCNN_BN_DO_32_4)

**Precision и recall**

In [ ]:
precision_classifier['FCNN_BN_DO_32_4'] = precision_score(
    y_test_resampled, y_test_pred_FCNN_BN_DO_32_4,
    average='macro', zero_division = np.nan)
recall_classifier['FCNN_BN_DO_32_4'] = recall_score(
    y_test_resampled, y_test_pred_FCNN_BN_DO_32_4,
    average='macro', zero_division = np.nan)

**F1**

In [ ]:
f1_classifier['FCNN_BN_DO_32_4'] = f1_score(
    y_test_resampled, y_test_pred_FCNN_BN_DO_32_4,
    average='macro', zero_division = np.nan)

**Значения метрик**

In [ ]:
print(f"accuracy - {accuracy_classifier['FCNN_BN_DO_32_4']*100:0.2f}%")
print(f"precision - {precision_classifier['FCNN_BN_DO_32_4']*100:0.2f}%")
print(f"recall - {recall_classifier['FCNN_BN_DO_32_4']*100:0.2f}%")
print(f"F1 - {f1_classifier['FCNN_BN_DO_32_4']*100:0.2f}%")

## Сравнение

In [ ]:
df = pd.DataFrame(
    [precision_classifier, recall_classifier, f1_classifier, accuracy_classifier],
    index=['Precision', 'Recall', 'F1-score', 'Accuracy'])
df

## Какие сигналы оказывают самое сильное влияние на ответ модели?

**SHAP**  

SHAP (**SHapley Additive exPlanations**) – это метод интерпретации модели, который **показывает вклад каждого признака в предсказание**.  

---

**Что означают SHAP-значения?**
SHAP value показывает, **насколько каждый признак изменяет предсказание модели** относительно базового уровня (среднего предсказания).  

- **Отрицательные SHAP-значения** → Признак снижает вероятность предсказанного класса  
- **Положительные SHAP-значения** → Признак увеличивает вероятность предсказанного класса  
- **Чем больше по модулю SHAP value**, тем **важнее** этот признак для предсказания  

In [ ]:
!pip install shap

In [ ]:
import shap

explainer = shap.Explainer(lambda x: FCNN_BN_32_4(
    torch.tensor(x.values, dtype=torch.float32)), X_train_resampled_scaled)
shap_values = explainer(X_test_resampled_scaled)

shap.summary_plot(shap_values,
                  X_test_resampled_scaled,
                  feature_names=[kks_to_description[kks] for kks in X_test_resampled_scaled.columns],
                  class_names=target_columns)